In [1]:
%pip install fireducks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.5 MB/s eta 0:00:00


In [1]:
import cv2
import os
# import fireducks.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import files

In [2]:
%pip install Google-Colab-Transfer

In [4]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from kaggle.api.kaggle_api_extended import KaggleApi

# api = KaggleApi()
# api.authenticate()

# # Replace 'username/dataset-name' with your dataset name
# api.dataset_download_files('subrahmanya090/arecanut', path='Arecanut-quality-classification/', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/subrahmanya090/arecanut


KeyboardInterrupt: 

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.


'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!kaggle datasets download -d subrahmanya090/arecanut -p /content/areca-nut

Dataset URL: https://www.kaggle.com/datasets/subrahmanya090/arecanut
License(s): apache-2.0
100% 4.68G/4.68G [00:53<00:00, 104MB/s] 
100% 4.68G/4.68G [00:53<00:00, 93.5MB/s]


In [7]:
!unzip /content/areca-nut/arecanut.zip -d /content/areca-nut/

Archive:  /content/areca-nut/arecanut.zip
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191315575.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191322886.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191828631.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191834963.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191846444.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191852083.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191858580.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191904151.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191909850.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191921179.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191939327.jpg  
  inflating: /content/areca-nut/arecanut/Bette/IMG_20241102_191952589.jpg  
  inflating: /content/areca-nut/arecanut/Bette

In [18]:
def preprocess_images(input_folder, output_folder):

    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            img = cv2.imread(input_path)

            if img is None:
                print(f"Skipping {filename}, unable to read the file.")
                continue

            resized_img = cv2.resize(img, (2048, 2048))

            gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
            blurred_img = cv2.GaussianBlur(gray_img, (5, 5), sigmaX=1.0, sigmaY=1.0)
            
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_img = clahe.apply(gray_img)

                                                                            # blurred_img = cv2.GaussianBlur(gray_img, (9, 9), 0)

                                                                            # source for sharpening images: https://medium.com/@sajjadhadi/mastering-opencv2-in-15-days-day-3-image-filtering-and-enhancement-ae1095f09aa5

            sharpening_kernel = np.array([[-1, -1, -1],
                             [-1, 9, -1],
                             [-1, -1, -1]])
            sharpened_image = cv2.filter2D(clahe_img, -1, sharpening_kernel)
            
            sobelx = cv2.Sobel(blurred_img, cv2.CV_64F, 1, 0, ksize=3)  # Gradient in X direction
            sobely = cv2.Sobel(blurred_img, cv2.CV_64F, 0, 1, ksize=3)  # Gradient in Y direction
            
            gradient_magnitude = np.hypot(sobelx, sobely)
            gradient_magnitude = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))
            

                                                                            # source for CLAHE (Contrast Limited Adaptive Histogram Equalization): https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

            normalized_img = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX)
            normalized_img = np.uint8(normalized_img)


            cv2.imwrite(output_path, normalized_img)

    print("Preprocessing complete. Images saved to:", output_folder)

#  Step1: 1stchali images preprocessing
output_folder='preprocessed/'
input_folder='input/'

#  Step2: Gotu images preprocessing
#  output_folder='gotupreprocessed'
#  input_folder='/content/areca-nut/arecanut/Gotu'

#  Step2: Gotu images preprocessing
#  output_folder='kolepreprocessed'
#  input_folder='/content/areca-nut/arecanut/Kole'

preprocess_images(input_folder,output_folder)



# def preprocess_images(input_folder, output_folder):

#     os.makedirs(output_folder, exist_ok=True)

#     for filename in os.listdir(input_folder):
#         if filename.endswith(('.jpg', '.jpeg', '.png')):
#             input_path = os.path.join(input_folder, filename)
#             output_path = os.path.join(output_folder, filename)

#             img = cv2.imread(input_path)

#             if img is None:
#                 print(f"Skipping {filename}, unable to read the file.")
#                 continue

#             resized_img = cv2.resize(img, (2048, 2048))

#             gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

#                                                                             # source for CLAHE (Contrast Limited Adaptive Histogram Equalization): https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

#             clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#             clahe_img = clahe.apply(gray_img)

#                                                                             # blurred_img = cv2.GaussianBlur(gray_img, (9, 9), 0)

#                                                                             # source for sharpening images: https://medium.com/@sajjadhadi/mastering-opencv2-in-15-days-day-3-image-filtering-and-enhancement-ae1095f09aa5

#             sharpening_kernel = np.array([[-1, -1, -1],
#                              [-1, 9, -1],
#                              [-1, -1, -1]])
#             sharpened_image = cv2.filter2D(clahe_img, -1, sharpening_kernel)

#             normalized_img = cv2.normalize(sharpened_image, None, 0, 255, cv2.NORM_MINMAX)
#             normalized_img = np.uint8(normalized_img)


#             cv2.imwrite(output_path, normalized_img)

#     print("Preprocessing complete. Images saved to:", output_folder)

# #  Step1: 1stchali images preprocessing
# output_folder='preprocessed'
# input_folder='example-image.jpg'

# #  Step2: Gotu images preprocessing
# #  output_folder='gotupreprocessed'
# #  input_folder='/content/areca-nut/arecanut/Gotu'

# #  Step2: Gotu images preprocessing
# #  output_folder='kolepreprocessed'
# #  input_folder='/content/areca-nut/arecanut/Kole'

# preprocess_images(input_folder,output_folder)

Preprocessing complete. Images saved to: preprocessed/


In [11]:
%pip install rembg 

  Using cached rembg-2.0.59-py3-none-any.whl.metadata (17 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached onnxruntime-1.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached PyMatting-1.1.13-py3-none-any.whl.metadata (7.5 kB)
  Using cached scikit_image-0.24.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached numba-0.60.0-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached imageio-2.36.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached rembg-2.0.59-py3-none-any.whl (39 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached onnxrunti

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'd:\\areca\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [ ]:
from rembg import remove
from PIL import Image

input_path = ''
output_path = ''

input = Image.open(input_path)
output = remove(input)
output.save(output_path)

In [27]:
import os
from rembg import remove
from PIL import Image

def remove_background_from_images(input_folder, output_folder):
   
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(input_folder):
        input_path = os.path.join(input_folder, file_name)
        
        if os.path.isfile(input_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
            try:
                with open(input_path, "rb") as img_file:  
                    input_data = img_file.read()
                    
                    output_data = remove(input_data)
                    
                    output_file_name = os.path.splitext(file_name)[0] + ".png"
                    output_path = os.path.join(output_folder, output_file_name)
                
                with open(output_path, "wb") as out_file:
                    out_file.write(output_data)
                
                print(f"Processed and saved: {output_path}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

remove_background_from_images("D:/Arecanut-quality-classification/arecanut-images/1stchali", "D:/Arecanut-quality-classification/arecanut-images")


Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191315575.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191322886.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191828631.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191834963.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191846444.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191852083.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191858580.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191904151.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191909850.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images\IMG_20241102_191921179.png
Processed 

In [28]:
remove_background_from_images("D:/Arecanut-quality-classification/arecanut-images/Gotu", "D:/Arecanut-quality-classification/arecanut-images/gpre")

Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103347688.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103359556.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103411575.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103424741.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103439343.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103459630.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103512746.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103519374.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/gpre\IMG_20241103_103525952.png
Processed and saved: D:/Arecanut-quality-classification/arecanut

In [29]:
remove_background_from_images("D:/Arecanut-quality-classification/arecanut-images/Kole", "D:/Arecanut-quality-classification/arecanut-images/kolepre")

Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172810420.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172834201.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172851808.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172901433.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172912057.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172922432.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172935029.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172944849.png
Processed and saved: D:/Arecanut-quality-classification/arecanut-images/kolepre\IMG_20241102_172953981.png
Processed and saved: D:/Arecanut-qual

In [23]:
#Step1: 1stchalipreprocessed images to zip
!zip -r 1stchali-preprocessed_images.zip /content/1stchalipreprocessed

#Step2: gotupreprocessed images to zip
!zip -r gotu-preprocessed_images.zip /content/gotupreprocessed

#Step3: kolepreprocessed images to zip
!zip -r kole-preprocessed_images.zip /content/kolepreprocessed

  adding: content/kolepreprocessed/ (stored 0%)
  adding: content/kolepreprocessed/IMG_20241102_191004414.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_181715574.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_181553081.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_190622252.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_173852733.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_184217998.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_182709447.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_174830992.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_182517352.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_185907619.jpg (deflated 0%)
  adding: content/kolepreprocessed/IMG_20241102_173129389.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_20241102_175657403.jpg (deflated 1%)
  adding: content/kolepreprocessed/IMG_2

In [18]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!cp stchali-preprocessed_images.zip /content/drive/MyDrive/
!cp gotu-preprocessed_images.zip /content/drive/MyDrive/
!cp kole-preprocessed_images.zip /content/drive/MyDrive/